In [53]:
import backtrader as bt
import pandas as pd
from datetime import datetime,timedelta

class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 7),
    )
    def log(self, txt, dt=None):
        ''' 记录'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
 
    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
 
        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
 
        # 增加移动平均指标
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)
        
        # 增加划线的指标
        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)
        bt.indicators.WeightedMovingAverage(self.datas[0], period=25,
                                            subplot=True)
        bt.indicators.StochasticSlow(self.datas[0])
        bt.indicators.MACDHisto(self.datas[0])
        rsi = bt.indicators.RSI(self.datas[0])
        bt.indicators.SmoothedMovingAverage(rsi, period=10)
        bt.indicators.ATR(self.datas[0], plot=False)
 
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
 
        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
 
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
 
            self.bar_executed = len(self)
 
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
 
        # Write down: no pending order
        self.order = None
 
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
 
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
 
    def next(self):
        # Simply log the closing price of the series from the reference
        # self.log('Close, %.2f' % self.dataclose[0])
 
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return 
        # Check if we are in the market
        if not self.position:
            # Not yet ... we MIGHT BUY if ...SMA均线
            if self.dataclose[0] > self.sma[0]: 
                # print(self.dataclose)
                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()
        else:
 
            if self.dataclose[0] < self.sma[0]:#SMA均线
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
 
                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [54]:
#获取数据
# stock_hfq_df = pd.read_excel("PRE_BCHAIN.xlsx",index_col='Date',parse_dates=True)
stock_hfq_df = pd.read_excel("Bitcoin.xlsx",index_col='date',parse_dates=True)
print(stock_hfq_df)
# stock_hfq_df = stock_hfq_df[0:60]
print(stock_hfq_df)
start_date = datetime(2016, 10, 21)  # 回测开始时间
# end_date = datetime(2021, 9, 10)  # 回测结束时间
# print(end_date)
end_date= start_date + timedelta(days=90)

                open     close    volume     Value_pre
date                                                  
2016-09-12    621.65    609.67    605600   -499.477377
2016-09-13    609.67    610.92    608000   -480.063230
2016-09-14    610.92    608.82    608600   -520.599902
2016-09-15    608.82    610.38    609500   -499.475845
2016-09-16    610.38    609.11    608100   -532.122381
...              ...       ...       ...           ...
2021-09-06  49947.38  51769.06  51769400  51356.824332
2021-09-07  51769.06  52677.40  52674200  51466.816991
2021-09-08  52677.40  46809.17  46774100  51521.913857
2021-09-09  46809.17  46078.38  46057600  51644.236339
2021-09-10  46078.38  46368.69  46385900  51718.689565

[1825 rows x 4 columns]
                open     close    volume     Value_pre
date                                                  
2016-09-12    621.65    609.67    605600   -499.477377
2016-09-13    609.67    610.92    608000   -480.063230
2016-09-14    610.92    608.82    608600

In [55]:
start_date = datetime(2016,11, 12)  # 回测开始时间
end_date = datetime(2021, 9, 10)  # 回测结束时间
# print(end_date)
# end_date= start_date + timedelta(days=360)
money =  800
# money = money*100
cerebro = bt.Cerebro()
# 增加一个策略
cerebro.addstrategy(TestStrategy)
# 将数据传入回测系统
data = bt.feeds.PandasData(dataname=stock_hfq_df, fromdate=start_date, todate=end_date)  # 加载数据
cerebro.adddata(data)
cerebro.broker.setcash(money)

cerebro.broker.setcommission(commission=0.02)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 800.00
2016-12-15, BUY CREATE, 776.75
2016-12-16, BUY EXECUTED, Price: 776.75, Cost: 776.75, Comm 15.54
2017-01-05, SELL CREATE, 994.67
2017-01-06, SELL EXECUTED, Price: 994.67, Cost: 776.75, Comm 19.89
2017-01-06, OPERATION PROFIT, GROSS 217.92, NET 182.49
2017-01-16, BUY CREATE, 830.50
2017-01-17, BUY EXECUTED, Price: 830.50, Cost: 830.50, Comm 16.61
2017-01-25, SELL CREATE, 893.05
2017-01-26, SELL EXECUTED, Price: 893.05, Cost: 830.50, Comm 17.86
2017-01-26, OPERATION PROFIT, GROSS 62.55, NET 28.07
2017-01-26, BUY CREATE, 915.96
2017-01-27, BUY EXECUTED, Price: 915.96, Cost: 915.96, Comm 18.32
2017-01-30, SELL CREATE, 912.19
2017-01-31, SELL EXECUTED, Price: 912.19, Cost: 915.96, Comm 18.24
2017-01-31, OPERATION PROFIT, GROSS -3.77, NET -40.33
2017-01-31, BUY CREATE, 964.71
2017-02-01, Order Canceled/Margin/Rejected
2017-02-01, BUY CREATE, 979.70
2017-02-02, Order Canceled/Margin/Rejected
2017-02-02, BUY CREATE, 983.42
2017-02-03, Order Canceled/Margin/Reje